# Process the data obtained from polarimetry and reorganize the data into the correct folders

In [1]:
%load_ext autoreload
%autoreload 2

import os
from processingmm import batch_processing

In [2]:
# set the parameters to run the script
directories = [r'C:\Users\romai\Documents\repositories\align_wavelenghts\data']
calib_directory = r'C:\Users\romai\Documents\repositories\align_wavelenghts\calib'

# set the parameters to be used for the line visualisation with the parameter parameter_set
parameter_set = 'TheoniPics'

# set run_all to true in order to run the pipeline on all the folders (even the ones already processed)
run_all = False

# PDDN_mode can be set to:
# 1. 'no': processes without using the PDDN
# 2. 'pddn': processes with PDDN when available (for 550nm and 650nm)
# 3. 'both': processes with PDDN when available and also without using the PDDN
PDDN_mode = 'both'

# Set the wavelengths to be processed
# 1. 'all': processes all the available wavelenght
# 2. [xxx, yyy]: processes only the wavelenghts 'xxx' and 'yyy'
wavelengths = 'all'

# Processing mode
# 1. 'full': processes everything (azimuth_noise, all plots and visualization) - 1x speed
# 2. 'no_visualization': processes azimuth noise + all plots but no visualization - 1.25x faster
# 3. 'fast': processes only the legacy plots (i.e. depolarization, azimuth, retardance and diattenuation) - 3x faster
processing_mode = 'no_visualization'

# Wether or not time mode (to get the processing times is activated or not)
time_mode = True

In [3]:
times, time_complete = batch_processing.batch_process_master(directories, calib_directory, run_all = run_all, 
                                parameter_set = parameter_set, PDDN = PDDN_mode, 
                                wavelengths = wavelengths, processing_mode = processing_mode,
                                time_mode = time_mode)
times

1. processing without PDDN...


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:31<00:00, 31.12s/it]


processing without PDDN done.

2. processing with PDDN.


  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

 >> PDDN Loading Model: C:\Users\romai\Documents\repositories\processingMM\src\processingmm\PDDN_model\PDDN_model_600_Fresh_HB.pt -- ...
 [wrn] PDDN Model Summary NOT Available!
 >> PDDN Loading Model: C:\Users\romai\Documents\repositories\processingMM\src\processingmm\PDDN_model\PDDN_model_600_Fresh_HB.pt -- Complete
*** PDDN DENOISING FILTER *** 
 >> PDDN Denoising: total time-points: 1000
 >> PDDN Denoising: start time-point: 1
 >> PDDN Denoising: final time-point: 0
 >> PDDN Denoising: recursive filtering enabled (2 x cascade): False
 >> MPI Denoising PDDN Performance: Elapsed time = 6.160 s
 
 >> Exported X3D .cod file as: ./temp_processing\2022-11-02_T_HORAO-59-AF_FR_S2_1\raw_data\600nm\600_Intensite_PDDN.cod
 >> PDDN Loading Model: C:\Users\romai\Documents\repositories\processingMM\src\processingmm\PDDN_model\PDDN_model_550_Fresh_HB.pt -- ...
 [wrn] PDDN Model Summary NOT Available!
 >> PDDN Loading Model: C:\Users\romai\Documents\repositories\processingMM\src\processingmm\PDDN_

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:37<00:00, 37.61s/it]

processing with PDDN done.


{'MM_processing': 0.4524860382080078,
 'azimuth_std_processing': 1.1858482360839844,
 'denoising': 0,
 'save_npz': 0.14338111877441406,
 'full_processing': 1.8546512126922607,
 'plotting': 23.361956119537354}

In [5]:
from processingmm import align_wavelengths
align_wavelengths.align_wavelenghts(directories, PDDN_mode, run_all)

 
 >> Exported X3D .cod file as: C:\Users\romai\Documents\repositories\align_wavelenghts\data\2022-10-11_T_HORAO-55-BF_FR_S1_1\raw_data\600nm\600_Intensite_PDDN_aligned.cod
 
 >> Exported X3D .cod file as: C:\Users\romai\Documents\repositories\align_wavelenghts\data\2022-10-11_T_HORAO-55-BF_FR_S1_1\raw_data\600nm\600_Intensite_aligned.cod
 
 >> Exported X3D .cod file as: C:\Users\romai\Documents\repositories\align_wavelenghts\data\2022-11-28_T_HORAO-63-AF_FR_S1_1\raw_data\600nm\600_Intensite_PDDN_aligned.cod
 
 >> Exported X3D .cod file as: C:\Users\romai\Documents\repositories\align_wavelenghts\data\2022-11-28_T_HORAO-63-AF_FR_S1_1\raw_data\600nm\600_Intensite_aligned.cod
